In [645]:
import sys

import pandas as pd

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import os
import time

In [646]:
REESTR = 'data/reestr_ci_ul.csv'
BASE_URL = "https://bo.nalog.ru/"
OUTPUT_DIR = '/Users/sand/DataspellProjects/finadvisor_git/data/bo.nalog.ru/'

In [647]:
# Если выходной папки нет - создать
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [648]:
def get_buh_page(driver, inn):
    '''
    Получение страницы бух отчетности по ИНН
    :param driver: chrome driver
    :param inn: ИНН
    :return: содержимое страницы
    '''
    # Load base page
    driver.get(BASE_URL)

    # Find inn input field
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'input-search'))
    )

    input_form = driver.find_element(By.CLASS_NAME, 'input-search')
    input_field = input_form.find_element(By.ID, 'search')
    input_field.clear()
    input_field.send_keys(inn)

    input_form.submit()

    # Find result report link
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'results-search-content'))
    )

    driver.execute_script("""
                let modal = document.querySelector('#modal');
                if(modal) {
                    modal.remove();
                }
            """)

    result_table = driver.find_element(By.CLASS_NAME, 'results-search-content')
    result_link = result_table.find_elements(By.TAG_NAME, 'a')

    result_link[0].click()

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'download-reports-wrapper'))
    )

    download_wrapper = driver.find_element(By.CLASS_NAME, 'download-reports-wrapper')
    button_link = download_wrapper.find_element(By.TAG_NAME, 'button')
    button_link.click()

    download_wrapper_report = download_wrapper.find_element(By.CLASS_NAME, 'download-reports')
    # Wait for modal is showing
    time.sleep(1)

    download_wrapper_buttons = download_wrapper_report.find_element(By.CLASS_NAME, 'download-reports-buttons')
    if download_wrapper_buttons is not None:
        button_link = download_wrapper_buttons.find_element(By.CLASS_NAME, 'button_link')
        if button_link is not None:
            button_link.click()

    button_md = download_wrapper_report.find_element(By.CLASS_NAME, 'button_md')
    button_md.click()



In [649]:
# Чтение реестра для списка ИНН
data = pd.read_csv(REESTR)
data.head()

,org_inn,name,short_name,type,category,subject_rf,date_msp,okved
0,2221111175,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ Т - ...,ООО Т - СОФТ,2,1,22,2018-08-10,62.01
1,2221122610,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ИЗДАТ...,ООО ИДК ЭНТРОН,2,1,22,2016-08-01,58
2,2221114426,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ДЖЕЙ...,ООО ДЖЕЙН ТУР,2,1,22,2016-08-01,79.90.1
3,2221112108,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ИЗОС...,ООО ИЗОСТУДИЯ НОВЫЙ ДЕНЬ,2,1,22,2016-08-01,73.1
4,2221119791,ЖИЛИЩНО-СТРОИТЕЛЬНЫЙ КООПЕРАТИВ ПИОНЕР,ЖСК ПИОНЕР,2,1,22,2019-08-10,94.99


In [650]:
# Чтобы не грузить повторно страницы - определяем уже загруженные
inns = [str(x) for x in data['org_inn'].values]
print('Всего организаций', len(inns))
loaded_inns = list(map(lambda x: x.split('.')[0], os.listdir(OUTPUT_DIR)))
print('Уже загружено', len(loaded_inns))
inns = set(inns) - set(loaded_inns)
print('Осталось загрузить', len(inns))

Всего организаций 108766
Уже загружено 3
Осталось загрузить 108766


In [651]:
# Open Chrome
PROXY = "123.57.36.131:80"
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--proxy-server=%s' % PROXY)
prefs = {"download.default_directory": OUTPUT_DIR}
chrome_options.add_experimental_option("prefs", prefs)
# chrome_options.add_argument(f"download.default_directory={OUTPUT_DIR}")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/sand/.wdm/drivers/chromedriver/mac64/98.0.4758.80/chromedriver] found in cache
<ipython-input-651-987318930a81>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [652]:
for inn in list(inns)[:30]:
    try:
        print(f'{inn}\t', end='')
        result = get_buh_page(driver, inn)
        print('success')
    except Exception as e:
        print('fail', file=sys.stdout)
        print(f'{inn} exception {str(e)}', file=sys.stderr)
    time.sleep(5)


1435338862	success
7801683256	success
278138160	success
1902027475	fail


1902027475 exception


6501108052	success
7743344526	success
2466277690	success
3849079180	success
7727362334	success
7726362437	success
7017442828	success
264077729	success
7841408182	success
6950088621	success
9703011404	success
7728354569	success
1215079411	success
5836674851	success
5262138245	success
7801337312	success
5032133891	success
7716582345	success
7202238760	success
8603239611	success
7708704705	success
7743344710	fail


7743344710 exception


7838090446	fail


7838090446 exception


7452060505	success
7707176403	success
6122010720	success


In [653]:
# Close Chrome
driver.close()
driver.quit()